# Prepare input CytoSPACE

In [1]:
global_path = './cell-cell-communication/' ## Path to the github downloaded repository

In [2]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sys
sys.path.insert(1, global_path + 'Functions/')
from process_visium import visium_noh5
patient = 'CID44971'
out_path = global_path + 'Data/Breast/Input_cytoSPACE/'


## Load data 

In [3]:
## Spatial data

main_path_exp = global_path + 'Data/Breast/spatial_data/filtered_count_matrices/'+patient+'_filtered_count_matrix/'
main_path_spatial = global_path + 'Data/Breast/spatial_data/spatial/'+patient+'_spatial/'
library_id = 'breast_tissue'

adata_vis = visium_noh5(main_path_exp, main_path_spatial, library_id)

## scRNA-seq data
adata = sc.read(global_path+'Data/Breast/normalized_log.h5ad')
adata = adata[adata.obs.Patient == patient]

## Save raw scRNA-seq counts as tsv file

In [4]:
counts = pd.DataFrame(adata.layers['counts'].toarray()).T
counts.columns = adata.obs.index
counts.index = adata.var.index
counts.index.name='GENES'
counts.to_csv(out_path + patient + '_GEX.tsv', sep = '\t')

## Add Celltype information

In [5]:
cell_label = pd.DataFrame(adata.obs.celltype_minor)
cell_label.columns = ['CellType']
cell_label.index.name='Cell IDs'
cell_label.to_csv(out_path + patient + '_cell_label.tsv', sep = '\t')

## Save raw spatial counts

In [6]:
counts_sp = pd.DataFrame(adata_vis.X.toarray()).T
counts_sp.columns = adata_vis.obs.index
counts_sp.index = adata_vis.var.index
counts_sp.index.name='V1'
counts_sp.to_csv(out_path + patient + '_GEX_sp.tsv', sep = '\t')

## Add coords

In [7]:
coords = adata_vis.obs[['array_row', 'array_col']]
coords.columns = ['row', 'col']
coords.index.name='SpotID'
coords.to_csv(out_path + patient +'_coords.tsv', sep = '\t')

## Run cytoSPACE using the terminal (bash)

In [ ]:
cytospace \
   -sp ./Data/Breast/Input_cytoSPACE/CID4535_GEX.tsv \
   -ctp ./Data/Breast/Input_cytoSPACE/CID4535_cell_label.tsv \
   -stp ./Data/Breast/Input_cytoSPACE/CID4535_GEX_sp.tsv \
   -cp ./Data/Breast/Input_cytoSPACE/CID4535_coords.tsv \
   -o ./Results/Breast/CytoSPACE/CID4535/